In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

zsh:1: command not found: apt-get
zsh:1: command not found: apt-get
tar: Error opening archive: Failed to open 'spark-3.2.3-bin-hadoop2.7.tgz'


Exception: Unable to find py4j in /content/spark-3.2.3-bin-hadoop2.7/python, your SPARK_HOME may not be configured correctly

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFilesx
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz
"), sep="\t", header=True, inferSchema=True)
df.show()

### Create DataFrames to match tables

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df = pd.DataFrame(df)

In [ ]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

In [ ]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(['product_id','product_title']).drop_duplicates()
products_df.show()

In [ ]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df = df.select(['review_id', 'customer_id' , 'product_id', 'product_parent', to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

In [ ]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()
     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/<database name>"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

mode = "append"
jdbc_url="jdbc:postgresql://mychallenge17.cnoe7lzdcxmt.us-east-1.rds.amazonaws.com:5432/mychallenge17"
config = {"user":"postgres", 
          "password": "Acc3ss12", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [ ]:
#Deliverable 2 

In [ ]:
# Confirm total_vote, helpful_votes, and star_rating are all integers
df

In [ ]:
# Filter wireless data frame on total_vote greater than or equal to 20
totalvotes_df = df.filter("total_votes>=20")
totalvotes_df.show()

In [ ]:
helpful_50_df = totalvotes_df.filter("helpful_votes/total_votes>=.50")
helpful_50_df.show()

In [ ]:
print(helpful_50_df.count())

In [ ]:
vine_review_df = helpful_50_df.filter(helpful_50_df["vine"] == "Y")
vine_review_df.show()

In [ ]:
novine_review_df = helpful_50_df.filter(helpful_50_df["vine"] == "N")
novine_review_df.show()

In [ ]:
from pyspark.sql.functions import count
# Total number of reviews for each dataframe from Step 3 and Step 4:
vine_review_count = vine_review_df.count()
print("Total Number of Reviews PAID and 'helpful': %f" % vine_review_count)

novine_review_count = novine_review_df.count()
print("Total Number of Reviews NON-PAID and 'helpful': %f" % novine_review_count)

In [ ]:
# Number of 5-star Reviews for above Paid and Non-Paid dataframes
star5_vine_df = vine_review_df.filter(novine_review_df["star_rating"]=="5")
print("Total Number of 5-star Reviews PAID and 'helpful': %f" % star5_vine_df.count())


star5_novine_df = novine_review_df.filter(novine_review_df["star_rating"]=="5")
print("Total Number of 5-star Reviews NON-PAID and 'helpful': %f" % star5_novine_df.count())

In [ ]:
# Percentage of 5-star Reviews for above Paid and Non-Paid dataframes
star5_vine_pct = (star5_vine_df.count()/vine_review_count)
print("Percent of 5-Star Wireless Reviews from PAID, 'helpful' dataset: %f" % star5_vine_pct)

star5_novine_pct = (star5_novine_df.count()/novine_review_count)
print("Percent of 5-Star Wireless Reviews from NON-PAID, 'helpful' dataset: %f" % star5_novine_pct)

In [ ]:
# Total number of all reviews
total_reviews_count = df.coun
print("Total Number of Wireless Reviews: %f" % total_reviews_count)

help_reviews_count = helpful_50_df.count()
print("Total Number of 'Helpful' Wireless Reviews: %f" % help_reviews_count)

In [ ]:
#Total number of 5-star reviews
star5_df = df.filter(df["star_rating"] == '5')
star5_df.show()

star5_count = star5_df.count()
print("Total Number of 5-Star Wireless Reviews: %f" % star5_count)

In [ ]:
#Total number of 5-star helpful reviews
star5_help_df = helpful_50_df.filter(helpful_50_df["star_rating"] == '5')
star5_help_df.show()

star5_help_count = star5_help_df.count()
print("Total Number of 5-Star 'Helpful' Wireless Reviews: %f" % star5_help_count)
     

In [ ]:
# Percentage of 5-star reviews paid vs non-paid 
# 5-star Paid reviews 
vine_star5_all_df = star5_df.filter(star5_df["vine"] == "Y")
vine_star5_all_count = vine_star5_all_df.count()

vine_star5_all_pct = (vine_star5_all_count/star5_count)
print("Number of 5-Star Wireless Reviews PAID: %f" % vine_star5_all_count)
print("Percent of 5-Star Wireless Reviews PAID: %f" % vine_star5_all_pct)
 
#5-star Non-Paid reviews
novine_star5_all_df = star5_df.filter(star5_df["vine"] == "N")
novine_star5_all_count = novine_star5_all_df.count()

novine_star5_all_pct = novine_star5_all_count/star5_count
print("Number of 5-Star Wireless Reviews NON-PAID: %f" % novine_star5_all_count)
print("Percent of 5-Star Wireless Reviews NON-PAID: %f" % novine_star5_all_pct)

In [ ]:
# Percentage of helpful 5-star reviews paid vs non-paid (% of 5 star reviews in Step 3 dataframe) 
# 5-star Paid helpful reviews 
vine_star5_help_df = star5_help_df.filter(star5_help_df["vine"] == "Y")
vine_star5_help_count = vine_star5_help_df.count()

vine_star5_help_pct = (vine_star5_help_count/star5_help_count)
print("Number of 5-Star, 'Helpful' Wireless Reviews PAID: %f" % vine_star5_help_count)
print("Percent of 5-Star,'Helpful'Wireless Reviews PAID: %f" % vine_star5_help_pct)
 
#5-star Non-Paid helpful reviews
novine_star5_help_df = star5_help_df.filter(star5_help_df["vine"] == "N")
novine_star5_help_count = novine_star5_help_df.count()

novine_star5_help_pct = novine_star5_help_count/star5_help_count
print("Number of 5-Star, 'Helpful' Wireless Reviews NON-PAID: %f" % novine_star5_help_count)
print("Percent of 5-Star,'Helpful'Wireless Reviews NON-PAID: %f" % novine_star5_help_pct)

In [ ]:
#Percent of Vine vs non-Vine all reviews
vine_all_df = df.filter(df["vine"] == "Y")
novine_all_df = df.filter(df["vine"] == "N")
vine_pct_all = (vine_all_df.count()/df.count())
novine_pct_all = (novine_all_df.count()/df.count())


print(df.count())
print(vine_all_df.count())
print(novine_all_df.count())
print(vine_pct_all)
print(novine_pct_all)

In [ ]:
# Percent of All Vine that is 5-Star
vine_all_star5_df = vine_all_df.filter(vine_all_df["star_rating"] == '5')
vine_all_star_df = vine_all_df.filter(vine_all_df["star_rating"] != '5')

novine_all_star5_df = novine_all_df.filter(novine_all_df["star_rating"] == '5')
novine_all_star_df = novine_all_df.filter(novine_all_df["star_rating"] != '5')

vine_pct_star5_all = (vine_all_star5_df.count()/vine_all_df.count())
novine_pct_star5_all = (novine_all_star5_df.count()/novine_all_df.count())

print(vine_all_star5_df.count())
print(vine_all_star_df.count())

print(novine_all_star5_df.count())
print(novine_all_star_df.count())

print(vine_pct_star5_all)
print(novine_pct_star5_all)

In [ ]:
# Pull out all Vine reviewed products
vine_prod_df = vine_all_df.select(vine_all_df["product_id"]).dropDuplicates()
vine_prod_df.show()
vine_prod_df.count()

In [ ]:
# create an inner join between all the Vine Reviews df and the full df
inner_join = vine_prod_df.join(df, vine_all_df.product_id == df.product_id, how="inner").dropDuplicates()
inner_join.show()
inner_join.count()

In [ ]:
# Rating breakdown 
new_df = inner_join.groupby("star_rating").count()
new_df.show()

In [ ]:
# Total Vine Reviews for Vine Products
vine_new_df = inner_join.filter(inner_join["vine"] == "Y")
vine_new_df.count()

In [ ]:
# Total Vine Reviews for Vine Products
novine_new_df = inner_join.filter(inner_join["vine"] == "N")
novine_new_df.count()
     

In [ ]:
# % of 5-star reviews are Vine or non-Vine Reviews
vine_star5_new = vine_new_df.groupby("star_rating").count()
vine_star5_new.show()

In [ ]:
# % of 5-star reviews are Vine or non-Vine Reviews
novine_star5_new = novine_new_df.groupby("star_rating").count()
novine_star5_new.show()